In [ ]:
# function file
from data_structures import PatientData
from sklearn.base import BaseEstimator
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from typing import Tuple
from sklearn.model_selection import train_test_split
import numpy as np
from dataclasses import dataclass
from typing import Dict, List
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from decoders import generate_pseudopopulations, DecodingResult
from multi_decoding import MultiResultsManager, plot_multi_patient_heatmap

In [ ]:
p566 = PatientData(pid='566')
p563 = PatientData(pid='563')
p562 = PatientData(pid='562')

In [ ]:
THRESHOLD = 0.1


p566_fr_neurons = p566.filter_neurons_by_fr(neurons=p566.neurons, window=(p566.times_dict['movie_start_rel'], p566.times_dict['preSleep_recall_start_rel']), threshold=THRESHOLD)
p566_mtl_fr_neurons = p566.filter_mtl_neurons(neurons=p566_fr_neurons)

p563_fr_neurons = p563.filter_neurons_by_fr(neurons=p563.neurons, window=(p563.times_dict['movie_start_rel'], p563.times_dict['preSleep_recall_start_rel']), threshold=THRESHOLD)
p563_mtl_fr_neurons = p563.filter_mtl_neurons(neurons=p563_fr_neurons)

p562_fr_neurons = p562.filter_neurons_by_fr(neurons=p562.neurons, window=(p562.times_dict['movie_start_rel'], p562.times_dict['preSleep_recall_start_rel']), threshold=THRESHOLD)
p562_mtl_fr_neurons = p562.filter_mtl_neurons(neurons=p562_fr_neurons)

neurons_list = [p562_mtl_fr_neurons, p563_mtl_fr_neurons, p566_mtl_fr_neurons]

In [ ]:
p562_mtl_fr_neurons

In [ ]:
concept_pairs_to_decode = []

selected_concepts = ['A.Amar',
  'A.Fayed',
  'B.Buchanan',
  'C.Manning',
  'C.OBrian',
  'J.Bauer',
  'K.Hayes',
  'M.OBrian',
  'N.Yassir',
  'R.Wallace',
  'T.Lennox',
]

for i, concept1 in enumerate(selected_concepts[:]):
    for concept2 in selected_concepts[i+1:]: #avoid duplicates and self-pairs
        concept_pairs_to_decode.append((concept1, concept2))

print(f"Number of concept pairs to decode: {len(concept_pairs_to_decode)}")
print(concept_pairs_to_decode[:3]) # Print first 5 pairs as example


In [ ]:
multi_mtl_manager = MultiResultsManager(
    patient_data_list=[p562, p563, p566],
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=True,
    neurons_list=[p562_mtl_fr_neurons, p563_mtl_fr_neurons, p566_mtl_fr_neurons]
)

In [ ]:
multi_mtl_manager.run_decoding_for_pairs(num_iter=5)


In [ ]:
best_concepts = [
    "A.Fayed", 
    "R.Wallace", 
    "T.Lennox", 
    "N.Yassir", 
    "K.Hayes", 
    "M.OBrian", 
    "J.Bauer", 
    "C.Manning"
]

In [ ]:
plot_multi_patient_heatmap(multi_mtl_manager, metric='test_accuracy', selected_concepts=best_concepts, show_numbers=False)
plt.suptitle('Multi-Patient Psuedopopulation Character Decoding Performance\nPatients: 562, 563, 566\nMTL neurons only, above 0.1Hz Firing Rate', fontsize=17)
#plt.savefig('mtl_multipatient_without_acc')
#


# All areas, neurons above 0.1hz decoding

In [ ]:
multi_all_manager = MultiResultsManager(
    patient_data_list=[p562, p563, p566],
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=True,
    neurons_list=[p562_fr_neurons, p563_fr_neurons, p566_fr_neurons]
)

In [ ]:
multi_all_manager.run_decoding_for_pairs(num_iter=5)


In [ ]:
plot_multi_patient_heatmap(multi_all_manager, metric='test_accuracy', selected_concepts=best_concepts, show_numbers=False, center=0.5)
plt.suptitle('Multi-Patient Psuedopopulation Character Decoding Performance\nPatients: 562, 563, 566\nAll neurons above 0.1Hz Firing Rate', fontsize=17)
#plt.savefig('multipatient_without_accuracy_full_spectrum')


## Figs for Soraya - MTL neurons, 0.1hz dropout, only best patient - p563, with pseudopopulations and without, 6x6 best concept and 3x3 best concepts


### with pseudopops:

In [ ]:
selected_concepts = [
    'A.Amar',
    'A.Fayed',
    'B.Buchanan',
    'C.Manning',
    'C.OBrian',
    'J.Bauer',
    'K.Hayes',
    'M.OBrian',
    'N.Yassir',
    'R.Wallace',
    'T.Lennox',
]
# Create all possible pairs of concepts
concept_pairs_to_decode = []
for i, concept1 in enumerate(selected_concepts):
    for concept2 in selected_concepts[i+1:]: # avoid duplicates and self-pairs
        concept_pairs_to_decode.append((concept1, concept2))

print(f"Number of concept pairs to decode: {len(concept_pairs_to_decode)}")



In [ ]:
p563_mtl_manager = MultiResultsManager(
    patient_data_list=[p563],  # Passing p563 as a single-item list
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=True,  # Using pseudopopulations
    neurons_list=[p563_mtl_fr_neurons]  # MTL neurons for p563
)

# Run the decoding with 5 iterations for more reliable results
p563_mtl_manager.run_decoding_for_pairs(num_iter=5)


In [ ]:
plot_multi_patient_heatmap(p563_mtl_manager, metric='test_accuracy', selected_concepts=best_concepts, show_numbers=True, center=0.5)


In [ ]:
best_six_concepts = [
    'A.Fayed',
    'T.Lennox',
    'M.OBrian',
    'K.Hayes',
    'J.Bauer',
    'N.Yassir'
]

best_three_concepts = [
    'A.Fayed',
    'T.Lennox',
    'M.OBrian'
]


In [ ]:
plot_multi_patient_heatmap(p563_mtl_manager, metric='test_accuracy', 
    selected_concepts=best_six_concepts, show_numbers=False, vmin=0, vmax=1.0, center=0.5)
plt.suptitle('Patient 563 Psuedopopulation 6 Characters Decoding Performance\nMTL neurons only, above 0.1Hz Firing Rate')
plt.savefig('563_mtl_pseudo_no_text_0-1')


In [ ]:
plot_multi_patient_heatmap(p563_mtl_manager, metric='test_accuracy', 
    selected_concepts=best_three_concepts, show_numbers=False, vmin=0, vmax=1.0, center=0.5)
plt.suptitle('Patient 563 Psuedopopulation 3 Characters Decoding Performance\nMTL neurons only, above 0.1Hz Firing Rate')
plt.savefig('563_mtl_pseudo_3chars_no_text_0-1')


Methods paragraph for Soraya
- character presence requires onset - off screen/not present to present. 
- for decoding pairs, we use exclusive onsets, character onsets where the other character being decoded is not present
- 1s bins starting 0.2 seconds after character onset in movie, corresponding to delay in MTL neuron response
- use pseudopopulations to augment data, even out number of classes - randomly sample train and test splits of each concept in the decoding pair to augment data size to 200 train and 60 test per concept in each decoding pair, so each decoder is trained on 400 and tested on 120 combined real and pseudopopulation responses.
- we use a linear SVM to decode between the characters to find the decoding hyperplane
- average performance across 5 iterations of the entire process, then generate heatmaps on average accuracy

We defined character presence by onset transitions from off-screen to on-screen, specifically utilizing exclusive onsets where the target character appeared while the comparison character remained absent. Neural activity was binned in 1-second windows beginning 200ms post-onset to capture the temporal dynamics of MTL responses. To address limited sample availability in human recordings, we employed pseudopopulation augmentation techniques, randomly sampling from observed neural responses to generate balanced datasets containing 200 training and 60 testing examples per character (400 training, 120 testing samples total per decoder). Character discrimination was performed using linear Support Vector Machines to establish optimal decoding hyperplanes between neural representations. Performance metrics were averaged across five independent iterations of the entire procedure to ensure statistical reliability. The resulting accuracy values were visualized as heatmaps displaying the discriminability between character representations in MTL neural populations.